#Set-up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 11.1 MB 5.3 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

#Scheduling for Gordon's Dining and Event Center

In [ ]:
import pandas as pd

In [ ]:
# reading csv file
df = pd.read_excel('input data.xlsx')
df

# extracting name column from data set
name_Cols = df['Name']

In [ ]:
name_Cols

In [ ]:
# monday = df[""].tolist()
df = df.iloc[:, 1:]
monday = df.values.tolist()
monday
len(monday)

249

In [ ]:
#inputs
num_workers = 249 #indexed with i
num_days = 7 #indexed with j
num_shifts = 3 #indexed with k
num_week = 1

min_workers_per_shift = 28
max_shift_perweek = 5
max_shifts_per_day = 1
min_shifts_per_worker = 3
max_closing_shifts = 2

# attemplted to create model on fewer num_workers as suggested by prof.
#num_working =[[0,1,0,0,1,0,1], [0,1,0,1,0,1,0], [1,0,1,0,1,0,0], [0,1,0,0,1,0,1], [0,1,0,1,0,1,0], [1,0,1,0,1,0,0], [0,0,1,1,1,0,0], [0,1,0,0,1,0,1], [0,1,0,1,0,1,0], [1,0,1,0,1,0,0], [0,1,0,0,1,0,1], [0,1,0,1,0,1,0], [1,0,1,0,1,0,0], [0,0,1,1,1,0,0]]
#num_working[0]
#len(num_working)
#len(num_working[0])






# Optimization

In [ ]:
for i, inner_list in enumerate(monday):
    for j, element in enumerate(inner_list):
      if element !=0:
        

#optimization model
        model = ConcreteModel() #create a model

#declare the decision variables
        model.x = Var(range(num_workers), range(num_days), range(num_shifts), domain=Binary) #model.x[i,j,k] is the DV for the ith worker on day j, shift k

#Constraint: Each shift requires at least 28 student employees
#for each day j
    #for each shift k
        #add a constraint that the sum across the student employees i >= 28
        model.shiftshaveminworkers = ConstraintList()
        for j in range(num_days): #for each day j
              for k in range(num_shifts): #for each shift k
                model.shiftshaveminworkers.add(expr = sum(model.x[i,j,k] for i in range(num_workers)) >= min_workers_per_shift)
        
#Constraint: no student employees works more than 1 shift per day
#for each student employees i
    #for each day j
        #add a cosntraint that the sum across the shifts k is <= 1
        model.maxshiftsperday = ConstraintList()
        for i in range(num_workers):
          for j in range(num_days):
            model.maxshiftsperday.add(expr = sum(model.x[i,j,k] for k in range(num_shifts)) <= max_shifts_per_day)

        model.maxshiftsperweek = ConstraintList()
        for i in range(num_workers):
          for j in range(num_week):
            model.maxshiftsperday.add(expr = sum(model.x[i,j,k] for k in range(num_shifts)) <= max_shift_perweek)

#Constraint: at least 5 shifts for each student employee
        model.minshifts = ConstraintList()
        for i in range(num_workers):
          model.minshifts.add(expr = sum(model.x[i,j,k] for j in range(num_days) for k in range(num_shifts)) >= min_shifts_per_worker)

#Constraint: at most 2 closing shifts per student employee
        model.closingshifts = ConstraintList()
        for i in range(num_workers):
          model.closingshifts.add(expr = sum(model.x[i,j,k] for j in range(num_days) if k == 2) <= max_closing_shifts)

        model.Objective = Objective(expr = sum(model.x[i,j,k] for i in range(num_workers) for j in range(num_days) for k in range(num_shifts)), sense = minimize)

        #model.pprint()

In [ ]:
print(f"element at index {i}, {j}: {element}")

element at index 248, 0: 1.0


In [ ]:
#Solving by using GLPK solver
opt = SolverFactory('glpk')
opt.options['mipgap'] = 0
results = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --mipgap 0 --write /tmp/tmpf1x_ygfv.glpk.raw --wglp /tmp/tmpgfp9xc4h.glpk.glp
 --cpxlp /tmp/tmpopcy8k20.pyomo.lp
Reading problem data from '/tmp/tmpopcy8k20.pyomo.lp'...
/tmp/tmpopcy8k20.pyomo.lp:36181: warning: lower bound of variable 'x1' redefined
/tmp/tmpopcy8k20.pyomo.lp:36181: warning: upper bound of variable 'x1' redefined
2512 rows, 5230 columns, 18178 non-zeros
5229 integer variables, all of which are binary
41410 lines were read
Writing problem data to '/tmp/tmpgfp9xc4h.glpk.glp'...
33665 lines were written
GLPK Integer Optimizer, v4.65
2512 rows, 5230 columns, 18178 non-zeros
5229 integer variables, all of which are binary
Preprocessing...
2262 rows, 5229 columns, 17430 non-zeros
5229 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 

In [ ]:
#printing out solution (with pandas)
print("total number of shifts assigned:", model.Objective())

total number of shifts assigned: 747.0


In [ ]:
schedule = [[[model.x[i,j,k]() for k in range(num_shifts)] for j in range(num_days)] for i in range(num_workers)]

result = pd.DataFrame(schedule, index = [i for i in name_Cols], columns = [f"day{i}" for i in range(num_days)]) 

In [ ]:
# Converting array into string to get the schedule on excel file because excel is 2 dimensional
def array_to_string(arr):
    return ' '.join(str(x) for x in arr)


result = result.applymap(array_to_string)
result

,day0,day1,day2,day3,day4,day5,day6
"Yap, Evelyn",1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0
"Agosto, Milo",0.0 0.0 1.0,0.0 0.0 0.0,0.0 0.0 1.0,0.0 0.0 0.0,0.0 1.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0
"Frantz, Hailey",1.0 0.0 0.0,1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0
"Higa, Everett",1.0 0.0 0.0,1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0
"Colón, Mya",1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,1.0 0.0 0.0,1.0 0.0 0.0
...,...,...,...,...,...,...,...
"Kathuria, Divij",1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 1.0
"Kashyap, Aishwarya",1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 1.0
"Zhao, Zixuan",1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 1.0
"Choong, Jing Wen",1.0 0.0 0.0,1.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 0.0,0.0 0.0 1.0


In [ ]:
# saving the optimized schedule in a csv file
result.to_csv('Final Schedule.csv', index=True)